In [0]:
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')
PATH_TO_DATA = Path('drive/My Drive/21_school/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
from datetime import date

In [0]:
train = pd.read_csv(PATH_TO_DATA / 'given/train.csv', parse_dates=['Contract termination date', 'Birth date'])
test = pd.read_csv(PATH_TO_DATA / 'given/test.csv', parse_dates=['Contract termination date', 'Birth date'])

In [0]:
train.sample(10)

,Birth date,Native city,Gender,Wave id,Level,Heard about school from,Life status,day_00,day_01,day_02,day_03,day_04,day_05,day_06,day_07,day_08,day_09,day_10,day_11,day_12,day_13,evalexpr,match_n_match,bsq,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Contract termination date,Memory entrance game,Logic entrance game,contract_status
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1283398074,1997-10-20,Москва,female,3,5.726415,from_21_student,study,5,0.0,0.0,10.0,30.0,5.0,70.0,30.0,0.0,0.0,50.0,-42.0,0.0,20.0,0.0,100.0,0.0,0.0,0.0,0.0,75.0,16.0,75.0,26.0,NaT,8,14,1
1688844938,1998-04-02,Москва,male,1,8.788961,NaN,NaN,0,NaN,35.0,30.0,0.0,5.0,0.0,0.0,0.0,6.0,-42.0,42.0,0.0,0.0,50.0,100.0,0.0,0.0,0.0,0.0,0.0,32.0,38.0,40.0,NaT,9,13,1
1745216633,1967-07-27,Москва,female,3,4.181818,other,other,10,NaN,0.0,30.0,5.0,15.0,20.0,35.0,0.0,0.0,40.0,17.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,32.0,36.0,NaT,6,10,1
50943425,1995-08-13,Оренбург,male,3,4.235294,from_friends,other,0,NaN,0.0,5.0,10.0,2.0,0.0,30.0,0.0,0.0,40.0,5.0,0.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,55.0,16.0,17.0,36.0,NaT,8,15,1
1105827193,1989-10-04,Москва,male,2,1.242188,NaN,NaN,0,NaN,10.0,0.0,0.0,10.0,0.0,0.0,0.0,15.0,35.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,11.0,27.0,NaT,8,10,1
614920839,1994-05-20,Moscow,male,1,8.185065,other,study,5,NaN,0.0,5.0,5.0,5.0,70.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,100.0,0.0,120.0,0.0,0.0,55.0,32.0,38.0,27.0,NaT,7,11,1
1456857303,1989-12-25,Санкт-Петербург,male,2,3.169697,NaN,NaN,10,0.0,0.0,5.0,10.0,0.0,70.0,0.0,0.0,0.0,50.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,59.0,59.0,40.0,NaT,7,7,1
973822652,1990-05-05,Екатеринбург,male,3,4.620321,from_friends,work,30,0.0,10.0,0.0,40.0,0.0,0.0,0.0,10.0,3.0,27.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,13.0,NaT,8,9,1
1535994441,1992-04-27,Москва,male,1,3.672727,other,other,30,10.0,20.0,0.0,20.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,100.0,0.0,5.0,0.0,0.0,0.0,32.0,27.0,18.0,2019-11-14 21:00:00+03:00,6,9,0


In [0]:
train.head()

,id,Birth date,Native city,Gender,Wave id,Level,Heard about school from,Life status,day_00,day_01,day_02,day_03,day_04,day_05,day_06,day_07,day_08,day_09,day_10,day_11,day_12,day_13,evalexpr,match_n_match,bsq,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Contract termination date,Memory entrance game,Logic entrance game,contract_status,Level_Wave id,Level_Heard about school from,Level_Life status,Level_day_07,Level_day_08,Level_match_n_match,Level_bsq,Level_rush_01,Level_rush_02,Level_exam_00,Level_Memory entrance game
0,129212391,1990-01-18,Пермь,male,3,7.463235,google_ads,work,5,NaN,0.0,80.0,0.0,1.0,0.0,15.0,10.0,0.0,50.0,21.0,20.0,55.0,81.0,0.0,0.0,0.0,0.0,0.0,75.0,43.0,100.0,44.0,NaT,9,11,1,7.463235294117648_3,7.463235294117648_google_ads,7.463235294117648_work,7.463235294117648_15.0,7.463235294117648_10.0,7.463235294117648_0.0,7.463235294117648_0.0,7.463235294117648_0.0,7.463235294117648_0.0,7.463235294117648_75.0,7.463235294117648_9
1,566688420,1996-07-31,Волгоград,female,1,8.785714,not added,NaN,0,NaN,10.0,45.0,5.0,1.0,35.0,0.0,5.0,16.0,0.0,9.0,0.0,10.0,0.0,0.0,0.0,110.0,0.0,0.0,0.0,16.0,22.0,22.0,NaT,7,10,1,8.785714285714286_1,8.785714285714286_not added,8.785714285714286_nan,8.785714285714286_0.0,8.785714285714286_5.0,8.785714285714286_0.0,8.785714285714286_0.0,8.785714285714286_0.0,8.785714285714286_0.0,8.785714285714286_0.0,8.785714285714286_7
2,242300495,1992-06-27,Новосибирск,female,2,6.775000,other,work,0,NaN,20.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,16.0,27.0,NaT,7,11,1,6.775_2,6.775_other,6.775_work,6.775_0.0,6.775_0.0,6.775_nan,6.775_0.0,6.775_nan,6.775_nan,6.775_0.0,6.775_7
3,715424753,1996-06-15,Тула,female,1,8.146104,not added,NaN,5,0.0,35.0,25.0,20.0,2.0,70.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,27.0,NaT,9,6,1,8.146103896103897_1,8.146103896103897_not added,8.146103896103897_nan,8.146103896103897_0.0,8.146103896103897_0.0,8.146103896103897_0.0,8.146103896103897_0.0,8.146103896103897_0.0,8.146103896103897_0.0,8.146103896103897_0.0,8.146103896103897_9
4,382884118,1968-07-18,Голицыно,male,3,4.272727,other,other,0,0.0,NaN,5.0,85.0,3.0,50.0,40.0,0.0,12.0,40.0,5.0,0.0,0.0,0.0,60.0,0.0,120.0,0.0,0.0,80.0,38.0,0.0,45.0,NaT,7,12,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
train['Native city'] = train['Native city'].fillna('Другой город')
test['Native city'] = test['Native city'].fillna('Другой город')

In [0]:
train['Heard about school from'] = train['Heard about school from'].fillna('not added')
test['Heard about school from'] = test['Heard about school from'].fillna('not added')

In [0]:
train['Life status'] = train['Life status'].fillna('not added')
test['Life status'] = test['Life status'].fillna('not added')

In [0]:
cols = []
for let in ['day', 'rush', 'exam']:
    cols += list(filter(lambda x: x[:len(let)] == let, train.columns))

cols += ['evalexpr', 'match_n_match', 'bsq']

In [0]:
for col in cols:
    train[col + '_na'] = train[col].isna().astype('int8')
    test[col + '_na'] = test[col].isna().astype('int8')

In [0]:
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

In [0]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

train['age'] = train['Birth date'].apply(calculate_age)
test['age'] = test['Birth date'].apply(calculate_age)

In [0]:
train['is_terminate_date'] = train['Contract termination date'].isna().astype('int8')
test['is_terminate_date'] = test['Contract termination date'].isna().astype('int8')

In [0]:
train['is_female_and_23-28'] = ((train['Gender'] == 'female') & ((train['age'] >= 23) | train['age'] <= 28)).astype('int8')
test['is_female_and_23-28'] = ((test['Gender'] == 'female') & ((test['age'] >= 23) | test['age'] <= 28)).astype('int8')

In [0]:
train['not_reg_count'] = train[[col + '_na' for col in cols]].sum(axis=1).values
test['not_reg_count'] = test[[col + '_na' for col in cols]].sum(axis=1).values

In [0]:
train['not_reg_mean'] = train[[col + '_na' for col in cols]].mean(axis=1).values
test['not_reg_mean'] = test[[col + '_na' for col in cols]].mean(axis=1).values

In [0]:
train['share_not_reg'] = train['not_reg_count'] / len(cols)
test['share_not_reg'] = test['not_reg_count'] / len(cols)

In [0]:
train['not_reg_std'] = train[[col + '_na' for col in cols]].std(axis=1).values
test['not_reg_std'] = test[[col + '_na' for col in cols]].std(axis=1).values

In [0]:
train['total_score_days'] = train[cols].sum(axis=1).values
test['total_score_days'] = test[cols].sum(axis=1).values

In [0]:
train['mean_score_days'] = train[cols].mean(axis=1).values
test['mean_score_days'] = test[cols].mean(axis=1).values

In [0]:
train['std_score_days'] = train[cols].std(axis=1).values
test['std_score_days'] = test[cols].std(axis=1).values

In [0]:
train.shape

(1060, 81)

In [0]:
test.shape

(706, 80)

In [0]:
train['Contract termination date'] = train['Contract termination date'].apply(lambda x: 1 if x else 0)
test['Contract termination date'] = test['Contract termination date'].apply(lambda x: 1 if x else 0)

In [0]:
train.isna().sum().sum()

0

In [0]:
train['is_level_greater_4'] = (train['Level'] > 4).astype('int8')
test['is_level_greater_4'] = (test['Level'] > 4).astype('int8')

In [0]:
test.shape

(706, 81)

In [0]:
bad_Moscow = train[train['Native city'].isin(['Москва ', 'Moscow',
                                                  'Москва  ', 'г. Москва',
                                                  'Mosocow', 'город Москва'])].index
train.loc[bad_Moscow, 'Native city'] = 'Москва'

bad_Peter = train[train['Native city'].isin(['Санкт-Петербург ', 'Saint Petersburg',
                                                 'Санкт-петербург'])].index
train.loc[bad_Peter, 'Native city'] = 'Санкт-Петербург'

cities_MO = ['Балашиха', 'Мытищи', 'Красногорск', 'Реутов', 'Химки', 'Долгопрудный', 'Люберцы',
             'Одинцово', 'Подольск', 'Жуковский', 'Лыткарино', 'Голицино', 'Королёв', 'Троицк',
             'Лобня', 'Иваново', 'Калуга', 'Дубна', 'Московская обл. г. Мытищи', 'Наро-Фоминск',
             'Королев']
cities_MO_index = train[train['Native city'].isin(cities_MO)].index
train.loc[cities_MO_index, 'Native city'] = 'МО'




bad_Moscow = test[test['Native city'].isin(['Москва ', 'Moscow',
                                                  'Москва  ', 'г. Москва',
                                                  'Mosocow', 'город Москва'])].index
test.loc[bad_Moscow, 'Native city'] = 'Москва'

bad_Peter = test[test['Native city'].isin(['Санкт-Петербург ', 'Saint Petersburg',
                                                 'Санкт-петербург'])].index
test.loc[bad_Peter, 'Native city'] = 'Санкт-Петербург'

cities_MO = ['Балашиха', 'Мытищи', 'Красногорск', 'Реутов', 'Химки', 'Долгопрудный', 'Люберцы',
             'Одинцово', 'Подольск', 'Жуковский', 'Лыткарино', 'Голицино', 'Королёв', 'Троицк',
             'Лобня', 'Иваново', 'Калуга', 'Дубна', 'Московская обл. г. Мытищи', 'Наро-Фоминск',
             'Королев']
cities_MO_index = test[test['Native city'].isin(cities_MO)].index
test.loc[cities_MO_index, 'Native city'] = 'МО'

In [0]:
train['Birth date'] = train['Birth date'].astype(int)/10**9
test['Birth date'] = test['Birth date'].astype(int)/10**9

In [0]:
train.to_pickle(PATH_TO_DATA / 'preprocessed/train.pkl')
test.to_pickle(PATH_TO_DATA / 'preprocessed/test.pkl')

In [0]:
test.shape

(706, 35)